# Data Cleaning


In [1]:
import pandas as pd
from matplotlib import pyplot as plt 
import numpy as np
import re

First, we load required data from reviews, and remove rows based on several criteria. 

In [2]:
# choosing only the columns that are needed. The column 'Review' contains the text that makes the dataframe heavy. So it is dropped.
reviews = pd.read_csv('../data/reviews.csv', usecols=['AuthorId', 'ReviewId', 'RecipeId', 'Rating'])
# Data analysis shows 'Rating'=0 is not neccesarily bad rating 
# (Refer to https://www.kaggle.com/code/gemmin/sentiment-analysis)
# Removing 0 ratings
reviews = reviews[reviews['Rating']!=0]
# Only reviews by active reviewers (selected_authors) are accepted, as the model will be based on active reviewers.
# Active reviewer is whoever has more than a minimum number of reviews in the data
review_counts = reviews.groupby('AuthorId').count()
selected_authors = review_counts.loc[review_counts['ReviewId']>=10].index.values
reviews = reviews.loc[reviews['AuthorId'].isin(selected_authors)]

print('number of selected reviews:', reviews['ReviewId'].nunique())
print('number of selected reviewers:', reviews['AuthorId'].nunique())
print('number of selected recipes:', reviews['RecipeId'].nunique())
 

number of selected reviews: 978879
number of selected reviewers: 14544
number of selected recipes: 238699


spliting train and test based on authors (the train and test datasets have different author lists)

In [23]:
import random
author_list = reviews['AuthorId'].unique()
random.shuffle(author_list)
split = len(author_list)//10
author_list_test, author_list_train = author_list[:split], author_list[split:]
train = reviews[reviews['AuthorId'].isin(author_list_train)]
test = reviews[reviews['AuthorId'].isin(author_list_test)]
train.to_csv('../data/reviews_train.csv')
test.to_csv('../data/reviews_test.csv')



creating train and test datasets

In [4]:
from sklearn.model_selection import train_test_split

reviews = reviews[['RecipeId', 'AuthorId', 'Rating']]
train, test = train_test_split(reviews, test_size=0.2)
train.to_csv('../data/reviews_train.csv')
test.to_csv('../data/reviews_test.csv')

Now the ratings are normalized based on the reviewer ratings statistics.

This normalization is not needed for KNN model of scikit-learn surprise library.


In [4]:
# substracting mean of reviewer's ratings
reviews['Rating'] -= reviews.groupby('AuthorId')['Rating'].transform(np.mean)
# deviding by std of reviewer's ratings
reviews['Rating'] /= reviews.groupby('AuthorId')['Rating'].transform(np.std)
reviews.describe()

,ReviewId,RecipeId,AuthorId,Rating
count,9.788790e+05,978879.000000,9.788790e+05,9.433290e+05
mean,7.080768e+05,153753.910811,1.264780e+07,1.108753e-17
std,4.470185e+05,128094.099809,1.541074e+08,9.931838e-01
min,7.000000e+00,38.000000,1.533000e+03,-3.465545e+01
25%,3.264205e+05,49047.000000,1.029370e+05,1.066013e-01
50%,6.674580e+05,112362.000000,2.275860e+05,3.495521e-01
75%,1.072974e+06,233848.000000,4.650560e+05,5.175935e-01
max,2.090332e+06,541030.000000,2.002872e+09,3.015113e+00


Sorting recipes based on the ratings and saving them

In [ ]:
recipes = pd.read_csv('../data/recipes.csv')
recipes = recipes.sort_values('ReviewCount', ascending=False)
recipes.to_csv('../data/recipes.csv')

creating a selected list of popular recipes with high review rate

In [5]:
def str2url(s: str) -> str:
    'returns a list of strings breaking the original string by "" '
    return re.search(r'"(.*?)"', s).group(0)[1:-1]

In [2]:
recipes = pd.read_csv('../data/recipes.csv')
# selecting ~ 10000 recipes
recipes_selected = recipes[(recipes['ReviewCount']>20) & (recipes['AggregatedRating']>=4)]
# removing recipes with Nan category and keywords
recipes_selected = recipes_selected.dropna(subset=['RecipeCategory', 'Keywords'])

# removing recipes with no images
recipe_ids = []
for ind in recipes_selected.index:
    recipe = recipes_selected.loc[ind]
    if re.search(r'"(.*?)"', recipe['Images'])==None:
        recipe_ids.append(ind)
recipes_selected = recipes_selected.drop(recipe_ids)
recipes_selected['Image'] = recipes_selected['Images'].map(str2url)

#recipes_selected.to_csv('../data/recipes_selected.csv')

Creating a light version of selected recipes with needed columns for streamlit app

In [1]:
import pandas as pd

cols = ['RecipeId', 'Name', 'Description', 'Image', 'RecipeCategory', 'Keywords',
       'AggregatedRating', 'ReviewCount', 'RecipeInstructions']
recipes = pd.read_csv('../data/recipes_selected.csv', usecols=cols, index_col='RecipeId')
recipes
#recipes.to_csv('../data/recipes_selected_summarized.csv')


,Name,Description,RecipeCategory,Keywords,AggregatedRating,ReviewCount,RecipeInstructions,Image
RecipeId,,,,,,,,
45809,Bourbon Chicken,I searched and finally found this recipe on th...,Chicken Breast,"c(""Chicken"", ""Poultry"", ""Meat"", ""Chinese"", ""As...",5.0,3063.0,"c(""Editor's Note: Named Bourbon Chicken becau...",https://img.sndimg.com/food/image/upload/w_555...
2886,Best Banana Bread,Make and share this Best Banana Bread recipe f...,Quick Breads,"c(""Breads"", ""Fruit"", ""Oven"", ""< 4 Hours"")",5.0,2273.0,"c(""Remove odd pots and pans from oven."", ""Preh...",https://img.sndimg.com/food/image/upload/w_555...
27208,To Die for Crock Pot Roast,"Amazing flavor, and so simple! No salt needed ...",One Dish Meal,"c(""Roast Beef"", ""Meat"", ""Kid Friendly"", ""Potlu...",5.0,1692.0,"c(""Place beef roast in crock pot."", ""Mix the d...",https://img.sndimg.com/food/image/upload/w_555...
89204,Crock-Pot Chicken With Black Beans &amp; Cream...,I love this Crock-Pot chicken recipe for two r...,One Dish Meal,"c(""Chicken Breast"", ""Chicken"", ""Poultry"", ""Cor...",4.5,1657.0,"c(""Take 4-5 frozen, yes, frozen, boneless chic...",https://img.sndimg.com/food/image/upload/w_555...
39087,Creamy Cajun Chicken Pasta,Make and share this Creamy Cajun Chicken Pasta...,Chicken Breast,"c(""Chicken"", ""Poultry"", ""Meat"", ""Cajun"", ""Kid ...",5.0,1586.0,"c(""Place chicken and Cajun seasoning in a bowl...",https://img.sndimg.com/food/image/upload/w_555...
...,...,...,...,...,...,...,...,...
150759,Deluxe Fried Eggs,A simple and delicious way to transform fried ...,Breakfast,"c(""Lunch/Snacks"", ""Onions"", ""Vegetable"", ""Aust...",5.0,21.0,"c(""Heat the oil and the lemon juice in a non-s...",https://img.sndimg.com/food/image/upload/w_555...
449167,Healthy Quinoa and Ground Turkey Stuffed Peppers,Make and share this Healthy Quinoa and Ground ...,Poultry,"c(""Vegetable"", ""Meat"", ""Weeknight"", ""< 4 Hours"")",5.0,21.0,"c(""Preheat oven to 350 degrees."", ""Combine qui...",https://img.sndimg.com/food/image/upload/w_555...
121399,Crock Pot Low Calorie Lemon Chicken,"A quick and easy, slow-cooked lemon chicken ma...",Chicken Breast,"c(""Chicken"", ""Poultry"", ""Vegetable"", ""Meat"", ""...",5.0,21.0,"c(""Coat the chicken with lemon-pepper seasonin...",https://img.sndimg.com/food/image/upload/w_555...
